<a href="https://colab.research.google.com/github/mattiapocci/PhilosopherRank/blob/master/NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#***Imports and Drive Mount***

In [1]:

!pip3 install -U spacy[cuda92]
!python3 -m spacy download en_core_web_lg
#https://stackoverflow.com/questions/54334304/spacy-cant-find-model-en-core-web-sm-on-windows-10-and-python-3-5-3-anacon
!python3 -m spacy link en_core_web_lg en_lg
!pip3 install fast-pagerank
import sys
import time
import os
import re
import numpy as np
import random
import json
import spacy
from spacy import displacy
import nltk
import pprint
from nltk.tokenize import word_tokenize
import time
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
nlp = spacy.load("en_lg")
spacy.prefer_gpu()
import numpy as np
from scipy import sparse

from fast_pagerank import pagerank
from fast_pagerank import pagerank_power

from google.colab import drive

drive.mount('/content/drive')


Requirement already up-to-date: spacy[cuda92] in /usr/local/lib/python3.6/dist-packages (2.2.4)
     |████████████████████████████████| 324.1MB 47kB/s 
     |████████████████████████████████| 827.9MB 1.2MB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.2.5-cp36-none-any.whl size=829180944 sha256=321c27213c495e89822fc3c4d5ec823982a0936c234cba6dc38cd9de0a5e9348
  Stored in directory: /tmp/pip-ephem-wheel-cache-sosu3h75/wheels/2a/c1/a6/fc7a877b1efca9bc6a089d6f506f16d3868408f9ff89f8dbfc
Successfully built en-core-web-lg
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_lg -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en_lg
You can now load the model via spacy.load('en_lg')
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk

# ***Testing Dataset***


In [0]:
testset = {}
phrases_influence=[["He was an influence of her",["her"],["He"]],
                   ["Jhon was an influence of Mark",["Mark"],["Jhon"]],
                   ["Milton was the main influence of Mallio and Tullio",["Mallio","Tullio"],["Milton"]],
                   ["Jhon's influence on Mark",["Mark"],["Jhon"]],
                   ["Markov's influence on Tito is the main reason of \"La Bella e la Bestia\", the main work of Tito's career. ",["Tito"],["Markov"]],
                   ["Jhon was Mark's influence",["Mark"],["Jhon"]],
                   ["At the university she was introduced to the writings of Aristotle and Plato, who would be her greatest influence and counter-influence respectively",["she"],["Aristotele", "Plato"]],
                   ["Jhon had influence on Mark's work",["Mark"],["Jhon"]],
                   ["Hegel had an important influence on all the different works of Einstein career",["Einstein"],["Hegel"]],
                   ["Mark cites Jhon as an influence",["Mark"],["Jhon"]],
                   ["Hegel cited Shopenauer as the main influence of his work",["Hegel"],["Shopenauer"]],
                   ["Jhon was cited by Mark as an influence",["Mark"],["Jhon"]],
                   ["Adler is cited as an influence on John Milton's testament",["Jhon", "Milton"],["Adler"]]]
phrases_influences=[["influences include him",[],["him"]],
                   ["influences include Jhon",[],["Jhon"]],
                   ["influences of his work include Jhon Milton and the author of  \"La Casa\", Rick Morty",[],["Jhon", "Milton","Rick","Morty"]],
                   ["Jhon and Adam were Mark's influences",["Mark"],["Jhon"]],
                   ["Aristotele, Anassagora and Pluto were Antichiola main influences",["Antichiola"],["Pluto","Aristotele","Anassagora"]],
                   ["Jhon and Adam were influences of Mark",["Mark"],["Jhon","Adam"]],
                   ["Macchiavelli and Dante Alighieri where the main influences of Boccaccio's Decameron",["Boccaccio"],["Macchiavelli","Dante", "Alighieri"]]]
phrases_influenced=[["He was influenced by him",["He"],["him"]],
                   ["Mark was influenced by Jhon",["Mark"],["Jhon"]],
                   ["Adler was also influenced by the philosophies of Immanuel Kant, Friedrich Nietzsche, Rudolf Virchow and the statesman Jan Smuts",["Adler"],["Immanuel", "Kant", "Friedrich", "Nietzsche", "Rudolf", "Virchow", "Jan", "Smuts"]],
                   ["Jhon has influenced Mark",["Mark"],["Jhon"]],
                   ["Macchiavelli with his work have deeply influenced the philosophies of Mathma Ghandi",["Mathma","Ghandi"],["Macchiavelli"]],
                   ["Jhon influenced Mark",["Mark"],["Jhon"]],
                   ["Anassagora influenced Pitagora on the ideas regarding geometry",["Pitagora"],["Anassagora"]],
                   ["Mark's work was influenced by Jhon",["Mark"],["Jhon"]],
                   ["Gandhi's \"Power to the Man\" was influenced by the medioeval thinker Mark Robben",["Ghandi"],["Mark", "Robben"]],
                   ["Jhon's work influenced Mark",["Mark"],["Jhon"]],
                   ["Marconi's work on the electromagnetic field influenced Einstein's ideas of photon's transmission",["Einstein"],["Marconi"]],
                   ["Jhon influenced Mark's work",["Mark"],["Jhon"]],
                   ["Pluto influenced the idea of time of Aristotele and Minos",["Aristotele", "Minos"],["Pluto"]]]
phrases_inspire=[
                    ["He seemed to inspire her",["her"],["He"]],
                    ["These revolutionizing ideas of Wang Yangming would later inspire prominent Japanese thinkers like Motoori Norinaga",["Mootori Norinaga"],["Wang Yangming"]],
                    ["John's work inspire Mark",["John"],["Mark"]]
                ]
phrases_inspiration=[
                        ["John was inspiration for Mark",["Mark"],["John"]],
                        ["Rozanov is the main source of inspiration for Dmitry Galkovsky",["Dmitry Galkovsky"],["Rozanov"]],
                        ["Jhon and Adam became inspiration for Mark",["Mark"],["John and Adam"]],
                        ["Jhon's work provided inspiration for Mark",["Mark"],["John"]],
                        ["He got the inspiration for this text from Schleiermacher ’ s Über die Religion ",["He"],["Schleiermacher"]],
                        ["Jhon's work was inspiration for Mark",["Mark"],["John"]],
                        ["While Murdoch 's thought is an inspiration for Conradi",["Conradi"],["Murdoch"]],
                        ["Jhon's work served as inspiration to Mark",["Mark"],["Jhon"]],
                        ["Lucian 's True Story inspired Cyrano de Bergerac , whose writings later served as inspiration for Jules Verne ",["Cyrano de Bergerac", "Jules Verne"],["Lucian"]],
                        ["Mark took inspiration from John",["Mark"],["John"]],
                        ["He also took inspiration from phenomenologist epistemology",["He"],["phenomenologist epistemology"]],
                        ["Mark drew inspiration from John", ["Mark"], ["John"]],
                        ["In particular , he drew inspiration from a Chinese Buddhist master named Tao-cho",["he"],["Tao-cho"]],
                        ["Mark provided inspiration to John",["John"],["Mark"]]
                    ]
phrases_inspired=[
                    ["He was inspired by him",["He"],["him"]],
                    ["Mark have inspired Jhon",["John"],["Mark"]],
                    ["Jhon had been inspired by Mark",["John"],["Mark"]],
                    ["In it , Petrarch claimed to have been inspired by Philip V of Macedon",["Petrarch"],["Philip V of Macedon"]],
                    ["Jhon's thinking was inspired by Mark",["John"],["Mark"]],
                    ["Newton 's work on infinite series was inspired by Simon Stevin 's decimals",["Newton"],["Simon Stevin"]],
                    ["Jhon's work was inspired by Marks",["John"],["Marks"]],
                    [ "Schiller was inspired by the play Julius of Tarent by Johann Anton Leisewitz .",["Schiller"],["Johann Anton Leisewitz"]],
                    [ "Mark was inspired by Jhon",["Mark"],["John"]],
                    ["As a youth , he was inspired by Mencius ’ proposition",["he"],["Mencius"]],
                    ["Jhon inspired Mark",["John"],["Mark"]],
                    [ "It also inspired him to take falsifiability as his criterion of demarcation between what is , and is not , genuinely scientific",["him"],["It"]],
                    ["Jhon inspired Mark's work",["Mark"],["John"]],
                    [ "Spinoza inspired the poet Shelley to write his essay",["Shelley"],["Spinoza"]]
                  ]

testset["phrases_influence"] = phrases_influence
testset["phrases_influenced"] = phrases_influenced
testset["phrases_influences"] = phrases_influences
testset["phrases_inspiration"] = phrases_inspiration
testset["phrases_inspired"] = phrases_inspired


#***Import JSON articles***


In [6]:
"""
{
    "philosopher": "name",
    "article": "plaintext_article",
    "pageid": "id",
    "table_influenced": 
        [
            "name_of_someone_philosopher_influenced_by"
        ]
    "table_influences":
        [

            "name_of_someone_philosopher_influences"
        ]
}
"""
#open the file
with open('/content/drive/My Drive/folder/result_3.json') as f:
  jsonlist = json.load(f)

FileNotFoundError: ignored

# ***Text Preprocessing***

In [0]:
"""
#HIM
#divide in sentences the article

#eliminate sentences that don't include influenc -e, -ed, -es, -ing etc

#pattern matching on patterns (see language analysis)

#Entity recognition and recostruction

#US
"mark influenced jhon. Today is friday"
#divide in sentences the article
"mark influenced jhon"
"Today is friday"
#eliminate sentences that don't include influenc -e, -ed, -es, etc
"mark influenced jhon"

#pattern matching on patterns (using spacy 
?)
[x = "mark", y = "jhon"]

#Entity recognition and recostruction(here we have a proble, when he had Hendrix he could )
"""

#pages that are errors
page_errors = ["13692155", "225052", "63948", "3072809"] 
start = time.time()
for elem in jsonlist:
    if elem["pageid"] in page_errors:
        continue

    #divide in sentences the article
    sent_list = nltk.sent_tokenize(elem["article"])
    
    sent_list = [word_tokenize(i) for i in sent_list]

    influence_list = []
    influence_declinations = ["influence", "influenced", "influences", "inspired", "inspiration"]


    for word_list in sent_list:
        temp = [x for x in word_list if x in influence_declinations]
        if len(temp) != 0:
            influence_list.append(' '.join(word for word in word_list))

    new_list = []
    for sent in influence_list:
        new_list.append(sent)
        #displacy.render(sent, style="dep", jupyter=True)
    elem["article"] = new_list
print(time.time() - start)


# ***La cosa piu logica da fare***

In [0]:
import re
def regex(phrase):
  left = []
  right = []
  influencers = []
  influenced = []

  try:
    left = re.split(r'have[\s].*[iI]nfluenced[\s.,]', phrase ,2)[0]
    right = re.split(r'have[\s].*[iI]nfluenced[\s.,]', phrase ,2)[1]
    influencers = re.findall(r'[A-Z][a-z\']+[\s.,]?', str(left))
    influenced = re.findall(r'[A-Z][a-z\']+[\s.,]?', str(right))
  except:
    try:
      left = re.split(r'was[\s].*[iI]nfluenced[\s.,]', phrase ,2)[0]
      right = re.split(r'was[\s].*[iI]nfluenced[\s.,]', phrase ,2)[1]
      influencers = re.findall(r'[A-Z][a-z\']+[\s.,]?', str(right))
      influenced = re.findall(r'[A-Z][a-z\']+[\s.,]?', str(left))
    except:
      try:
        left = re.split(r'[iI]nfluenced[\s]by[\s.,]', phrase ,2)[0]
        right = re.split(r'[iI]nfluenced[\s]by[\s.,]', phrase ,2)[1]
        influencers = re.findall(r'[A-Z][a-z\']+[\s.,]?', str(right))
        influenced = re.findall(r'[A-Z][a-z\']+[\s.,]?', str(left))
      except:
        try:
          left = re.split(r'has[\s][iI]nfluenced[\s.,]', phrase ,2)[0]
          right = re.split(r'has[\s][iI]nfluenced[\s.,]', phrase ,2)[1]
          influencers = re.findall(r'[A-Z][a-z\']+[\s.,]?', str(left))
          influenced = re.findall(r'[A-Z][a-z\']+[\s.,]?', str(right))
        except:
          try:
            left = re.split(r'[iI]nfluenced[\s.,]', phrase ,2)[0]
            right = re.split(r'[iI]nfluenced[\s.,]', phrase ,2)[1]
            influencers = re.findall(r'[A-Z][a-z\']+[\s.,]?', str(left))
            influenced = re.findall(r'[A-Z][a-z\']+[\s.,]?', str(right))
          except:
            try:
              left = re.split(r'[iI]nfluences?[\s].*on[\s.,]', phrase ,2)[0]
              right = re.split(r'[iI]nfluences?[\s].*on[\s.,]', phrase ,2)[1]
              influencers = re.findall(r'[A-Z][a-z\']+[\s.,]?', str(left))
              influenced = re.findall(r'[A-Z][a-z\']+[\s.,]?', str(right))
            except:
              try:
                left = re.split(r'[iI]nfluences?[\s].*of[\s.,]', phrase ,2)[0]
                right = re.split(r'[iI]nfluences?[\s].*of[\s.,]', phrase ,2)[1]
                influencers = re.findall(r'[A-Z][a-z\']+[\s.,]?', str(left))
                influenced = re.findall(r'[A-Z][a-z\']+[\s.,]?', str(right))
              except:
                try:
                  left = re.split(r'[iI]nfluences?[\s].*include[\s.,]', phrase ,2)[0]
                  right = re.split(r'[iI]nfluences?[\s].*include[\s.,]', phrase ,2)[1]
                  influencers = re.findall(r'[A-Z][a-z\']+[\s.,]?', str(right))
                  influenced = re.findall(r'[A-Z][a-z\']+[\s.,]?', str(left))
                except:
                  influencers = []
                  influenced = []
  print('Phrase: ' + phrase)
  print('Influencers: ' + str(influencers))
  print('Influenced: ' + str(influenced)+'\n')
  return(influenced,influencers)

# ***Testing Mattia***

In [0]:
for phrase in phrases_influences:
  regex(phrase)

# ***Processing Giuliano(Spacy Dependencies)***

In [0]:

"""
------------------HIS APPROACH-------------------------
1. X cite(s|d) Y1, Y2, …, Yn as (an) influence(s)
2. X was influenced by Y1, Y2, …, Yn
3. Y has been cited as an influence by X1, X2, …, Xn
4. Y influence on X1, X2, …, Xn …
5. Y1, Y2, …, Yn influenced hi(m|s) …


------------------OUR APPROACH-------------------------
Influence:
X be an influence (of/on/for/to) Y 
X's influence (of/on/for/to) Y
X's have influence on Y
X be Y's influence
Y cite X as influence
X was cited as influence by Y


influences:
influences (include/of) X
X be Y's influences
X and X were influences of Y


influencing:
none


influenced:
Y be influenced by X
X have influenced Y
X influenced Y
Y's thinking was influenced by X
X influenced Y's work
X's work influenced Y


X ispiratore Y ispirante
inspired
Y be influenced by X
X have inspired Y
Y had been inspired by X
Y's thinking/work was inspired by X
Y was inspired by X
X inspired Y


inspire
none


inspiration
X became/was inspiration for Y
X's work provided/was inspiration for Y
X's work served as inspiration to Y
Y took/drew inspiration from X
X provided inspiration to Y

"""

def is_verb(phrase):
    res = re.search(r'influenced\b', phrase) or re.search(r'inspired\b', phrase)
    return res is not None



"""
Prints dependencies and displacy visuals for a phrase
Params: sent, a sentence
Returns: -
"""
def get_info(sent):
    print(sent)
    doc = nlp(sent)
    displacy.render(doc, style="dep", jupyter=True)  
    for token in doc:
        print(token.text, token.dep_, token.head.text, token.head.pos_,
                [child for child in token.children])



"""
Spacy processing
Params: phrase, a phrase
Returns:    influenced, list of influenced chunks
            influencers, list of influencers chunks
"""
def process_spacy(phrase):
    if is_verb(phrase):
        return process_verb(phrase)
    else:
        return process_name(phrase)


"""
Process influence or inspiration phrase
Params: phrase, a phrase tagged "influence" or "inspiration"
Returns:    influenced, list of influenced chunks
            influencers, list of influencers chunks

"""
#add climbing three for the subj
def process_name(phrase):
    influenced = []
    influencers = []
    name = ["influence", "inspiration", "influences"]
    doc = nlp(phrase)
    compounds = []
    for token in doc:
        #è un soggetto passivo, ex jhon is cited as an influence 
        if token.dep_ == "nsubjpass" and token.head.pos_ == "VERB" :
            influencers.append(token.text)

        #è un soggetto attivo, ex mark was an influence
        if token.dep_ == "nsubj" and token.head.pos_ == "AUX":
            influencers.append(token.text)

        #è un soggetto attivo, ex mark cited jhon as an influence
        if token.dep_ == "nsubj" and token.head.pos_ == "VERB":
            influenced.append(token.text)
        
        #è un oggetto
        if token.dep_ in ["pobj", "conj", "appos", "dobj"]:
            flag = 0
            tok = token.text
            while flag == 0:
                if token.head.text in name:
                    #see if it is passive or active
                    if "auxpass" in [x.dep_ for x in token.head.children]:
                        influencers.append(tok)
                    else:
                        influenced.append(tok)
                    flag = 1
                elif token.head.text == "cited" or token.head.text == "cites":
                    if token.dep_ == "agent":
                        influenced.append(tok)
                    else:
                        influencers.append(tok)
                    flag = 1
                elif token.head.pos_ == "VERB":
                    if "influence" in [x.text for x in token.head.children] or "influences" in [x.text for x in token.head.children] or "inspiration" in [x.text for x in token.head.children]:
                        print("ok")
                        influencers.append(tok)
                    flag = 1
                elif token.head.dep_ in ["prep", "agent", "conj", "appos", "dobj", "pobj"]:
                    token = token.head
                elif token.head.dep_ in ["nsubj", "nsubjpass"]:
                    #è un soggetto passivo, ex jhon is cited as an influence 
                    if token.dep_ == "nsubjpass" and token.head.pos_ == "VERB" :
                        influencers.append(token.text)

                    #è un soggetto attivo, ex mark was an influence
                    if token.dep_ == "nsubj" and token.head.pos_ == "AUX":
                        influencers.append(token.text)

                    #è un soggetto attivo, ex mark cited jhon as an influence
                    if token.dep_ == "nsubj" and token.head.pos_ == "VERB":
                        influenced.append(token.text)
                    
                    flag = 1
                else:
                    flag = 1

                
                
        #è un genitivo
        if token.dep_ == "poss":
            tok = token.text
            token = token.head
            flag = 0
            while flag == 0:
                if token.text in name:
                    if token.dep_ in ["attr", "pobj", "dobj"]:
                        influenced.append(tok)
                    else:
                        influencers.append(tok)
                    flag = 1
                elif token.head.dep_ in ["prep", "agent", "conj", "appos", "dobj", "pobj"]:
                    token = token.head
                else:
                    flag = 1

            
                
        #catch compounds
        if token.dep_ == "compound":
            compounds.append(token)

    #look for compound names
    for token in compounds:
        if token.head.text in influenced:
            influenced.append(token.text)
        if token.head.text in influencers:
            influencers.append(token.text)
    return influenced, influencers




"""
Process influenced or inspired phrase
Params: phrase, a phrase tagged "influencer" or "inspired"
Returns:    influenced, list of influenced chunks
            influencers, list of influencers chunks

"""

#aggiungi compound e influenced
#add climbing three for the subj
def process_verb(phrase):
    influenced = []
    influencers = []
    doc = nlp(phrase)
    verbs = ["influenced","inspired"]
    compounds = []
    for token in doc:
        #SOGGETTO
        #è un soggetto passivo, ex Mark was infuenced
        if token.dep_ == "nsubjpass" and token.head.text in verbs :
            influenced.append(token.text)

        #è un soggetto attivo, ex Jhon influenced
        if token.dep_ == "nsubj":
            if token.head.text in verbs:
                influencers.append(token.text)
            for ch in token.head.children:
                if "auxpass" in [x.dep_ for x in ch.children] and ch.text in verbs:
                    influenced.append(token.text)
                elif ch.text in verbs:
                    influencers.append(token.text)



        #è un oggetto
        if token.dep_ in ["pobj", "conj", "appos", "dobj"]:
            flag = 0
            tok = token.text
            while flag == 0:
                if token.head.dep_ in ["prep", "agent", "conj", "appos", "dobj", "pobj"]:
                    token = token.head
                elif token.head.text in verbs:
                    #see if it is passive or active
                    if "auxpass" in [x.dep_ for x in token.head.children]:
                        influencers.append(tok)
                    else:
                        influenced.append(tok)
                    flag = 1
                elif token.head.dep_ in ["nsubj", "nsubjpass"]:

                    token = token.head
                    if token.dep_ == "nsubjpass" and token.head.text in verbs :
                        influenced.append(tok)

                    #è un soggetto attivo, ex Jhon influenced
                    if token.dep_ == "nsubj":
                        if token.head.text in verbs:
                            influencers.append(tok)
                        for ch in token.head.children:
                            if "auxpass" in [x.dep_ for x in ch.children] and ch.text in verbs:
                                influenced.append(tok)
                            elif ch.text in verbs:
                                influencers.append(tok) 
                    flag = 1                   
                else:
                    flag = 1
        

        #è un genitivo
        if token.dep_ == "poss":
            tok = token.text
            token = token.head
            #di un soggetto
            if token.dep_== "nsubjpass" and token.head.text in verbs :
                influenced.append(tok)
            if token.dep_ == "nsubj" and token.head.text in verbs:
                influencers.append(tok)
            #di un oggetto
            if token.dep_ in ["pobj", "conj", "appos", "dobj"]:
                flag = 0
                while flag == 0:
                    if token.head.dep_ in ["prep", "agent", "conj", "appos", "dobj", "pobj"]:
                        token = token.head
                    elif token.head.text in verbs:
                        #see if it is passive or active
                        if "auxpass" in [x.dep_ for x in token.head.children]:
                            influencers.append(tok)
                        else:
                            influenced.append(tok)
                        flag = 1
                    else:
                        flag = 1


        #catch compounds
        if token.dep_ == "compound":
            compounds.append(token)

    #look for compound names
    for token in compounds:
        if token.head.text in influenced:
            influenced.append(token.text)
        if token.head.text in influencers:
            influencers.append(token.text)
    return influenced, influencers
    


# ***Testing Giuliano***

In [28]:
"""
for phr in phrases_influenced:
    get_info(phr[0])
    influenced, influencer = process_verb(phr[0])
    print("______EVAL______")
    print("influenced = ",phr[1], "influencers", phr[2])
    print("influenced = ",influenced, "influencers", influencer)
    print(len([x for x in influenced if x in phr[1]]), "on", len(phr[1]))
    print(len([x for x in influencer if x in phr[2]]), "on", len(phr[2]))

print("--------------------------------------------------------------------------------")
for phr in phrases_inspired:
    get_info(phr[0])
    influenced, influencer = process_verb(phr[0])
    print("______EVAL______")
    print("influenced = ",phr[1], "influencers", phr[2])
    print("influenced = ",influenced, "influencers", influencer)
    print(len([x for x in influenced if x in phr[1]]), "on", len(phr[1]))
    print(len([x for x in influencer if x in phr[2]]), "on", len(phr[2]))
"""

for phr in phrases_influence:
    get_info(phr[0])
    influenced, influencer = process_name(phr[0])
    print("______EVAL______")
    print("influenced = ",phr[1], "influencers", phr[2])
    print("influenced = ",influenced, "influencers", influencer)
    print(len([x for x in influenced if x in phr[1]]), "on", len(phr[1]))
    print(len([x for x in influencer if x in phr[2]]), "on", len(phr[2]))

print("--------------------------------------------------------------------------------")
for phr in phrases_influences:
    get_info(phr[0])
    influenced, influencer = process_name(phr[0])
    print("______EVAL______")
    print("influenced = ",phr[1], "influencers", phr[2])
    print("influenced = ",influenced, "influencers", influencer)
    print(len([x for x in influenced if x in phr[1]]), "on", len(phr[1]))
    print(len([x for x in influencer if x in phr[2]]), "on", len(phr[2]))

print("--------------------------------------------------------------------------------")
for phr in phrases_inspiration:
    get_info(phr[0])
    influenced, influencer = process_name(phr[0])
    print("______EVAL______")
    print("influenced = ",phr[1], "influencers", phr[2])
    print("influenced = ",influenced, "influencers", influencer)
    print(len([x for x in influenced if x in phr[1]]), "on", len(phr[1]))
    print(len([x for x in influencer if x in phr[2]]), "on", len(phr[2]))



He was an influence of her


He nsubj was AUX []
was ROOT was AUX [He, influence]
an det influence NOUN []
influence attr was AUX [an, of]
of prep influence NOUN [her]
her pobj of ADP []
______EVAL______
influenced =  ['her'] influencers ['He']
influenced =  ['her'] influencers ['He']
1 on 1
1 on 1
Jhon was an influence of Mark


Jhon nsubj was AUX []
was ROOT was AUX [Jhon, influence]
an det influence NOUN []
influence attr was AUX [an, of]
of prep influence NOUN [Mark]
Mark pobj of ADP []
______EVAL______
influenced =  ['Mark'] influencers ['Jhon']
influenced =  ['Mark'] influencers ['Jhon']
1 on 1
1 on 1
Milton was the main influence of Mallio and Tullio


Milton nsubj was AUX []
was ROOT was AUX [Milton, influence]
the det influence NOUN []
main amod influence NOUN []
influence attr was AUX [the, main, of]
of prep influence NOUN [Mallio]
Mallio pobj of ADP [and, Tullio]
and cc Mallio PROPN []
Tullio conj Mallio PROPN []
______EVAL______
influenced =  ['Mallio', 'Tullio'] influencers ['Milton']
influenced =  ['Mallio', 'Tullio'] influencers ['Milton']
2 on 2
1 on 1
Jhon's influence on Mark


Jhon poss influence NOUN ['s]
's case Jhon PROPN []
influence ROOT influence NOUN [Jhon, on]
on prep influence NOUN [Mark]
Mark pobj on ADP []
______EVAL______
influenced =  ['Mark'] influencers ['Jhon']
influenced =  ['Mark'] influencers ['Jhon']
1 on 1
1 on 1
Markov's influence on Tito is the main reason of "La Bella e la Bestia", the main work of Tito's career. 


...

Mark nsubj took VERB []
took ROOT took VERB [Mark, inspiration, from]
inspiration dobj took VERB []
from prep took VERB [John]
John pobj from ADP []
ok
ok
______EVAL______
influenced =  ['Mark'] influencers ['John']
influenced =  ['Mark'] influencers ['inspiration', 'John']
1 on 1
1 on 1
He also took inspiration from phenomenologist epistemology


He nsubj took VERB []
also advmod took VERB []
took ROOT took VERB [He, also, inspiration, from]
inspiration dobj took VERB []
from prep took VERB [epistemology]
phenomenologist compound epistemology NOUN []
epistemology pobj from ADP [phenomenologist]
ok
ok
______EVAL______
influenced =  ['He'] influencers ['phenomenologist epistemology']
influenced =  ['He'] influencers ['inspiration', 'epistemology', 'phenomenologist']
1 on 1
0 on 1
Mark drew inspiration from John


Mark nsubj drew VERB []
drew ROOT drew VERB [Mark, inspiration, from]
inspiration dobj drew VERB []
from prep drew VERB [John]
John pobj from ADP []
ok
ok
______EVAL______
influenced =  ['Mark'] influencers ['John']
influenced =  ['Mark'] influencers ['inspiration', 'John']
1 on 1
1 on 1
In particular , he drew inspiration from a Chinese Buddhist master named Tao-cho


In prep drew VERB [particular]
particular amod In ADP []
, punct drew VERB []
he nsubj drew VERB []
drew ROOT drew VERB [In, ,, he, inspiration, from]
inspiration dobj drew VERB []
from prep drew VERB [master]
a det master NOUN []
Chinese amod master NOUN []
Buddhist compound master NOUN []
master pobj from ADP [a, Chinese, Buddhist, named]
named acl master NOUN [cho]
Tao compound cho PROPN []
- punct cho PROPN []
cho oprd named VERB [Tao, -]
ok
ok
______EVAL______
influenced =  ['he'] influencers ['Tao-cho']
influenced =  ['he'] influencers ['inspiration', 'master', 'Buddhist']
1 on 1
0 on 1
Mark provided inspiration to John


Mark nsubj provided VERB []
provided ROOT provided VERB [Mark, inspiration, to]
inspiration dobj provided VERB []
to dative provided VERB [John]
John pobj to ADP []
ok
______EVAL______
influenced =  ['John'] influencers ['Mark']
influenced =  ['Mark'] influencers ['inspiration']
0 on 1
0 on 1


# ***Create Processed Json***

In [0]:

"""
Input cella:
jsonlist = 
[
    {
        "philosopher" = "philosopher_name",
        "pageid" = id,
        "article" = 
        [
            "frase contenente influence",

            "frase contenente influence", etc
        ]
    }
]

Output cella:
output =
[
    {
        "philosopher" : "philosopher_name",
        "pageid" : id,
        "article" : 
            [
                "frase contenente influence",

                "frase contenente influence" 
            ]

        "rich_article" :
            [
                {
                    'influenced': ['East ', 'Asia '],
                    'influencers': ['Influence ', 'Zhuangzi ',"Plato", "William", "Ralph", "Inge"],
                    'phrase': '== Influence == Zhuangzi has influenced '
                                'thinking far beyond East Asia .'
                }
            ]
    }

]
"""
#some prints and infos

#pprint.pprint(jsonlist)
print(len(jsonlist))
jsonlist_no_empty = [i for i in jsonlist if len(i["article"]) != 0]
print(len(jsonlist_no_empty))
#pprint.pprint(jsonlist_no_empty)

jsonlist_processed = jsonlist_no_empty

for phil in jsonlist_processed:
    rich_article = []
    for phrase in phil["article"]:
        #use regex for regex processing or process_spacy for spacy processing
        influenced, influencers = process_spacy(phrase)
        elem = {}
        elem["influenced"] = influenced
        elem["influencers"] = influencers
        elem["phrase"] = phrase
        rich_article.append(elem)
    phil["rich_article"] = rich_article

#pprint.pprint(jsonlist_processed)

# ***Cleaning and Recostruction(Luigi)***

In [0]:

"""
input =
[
    {
        "philosopher" : "philosopher_name",
        "pageid" : id,
        "article" : 
            [
                "frase contenente influence",

                "frase contenente influence" 
            ]

        "rich_article" :
            [
                {
                    'influenced': ['East ', 'Asia '],
                    'influencers': ['Influence ', 'Zhuangzi ',"Plato", "William", "Ralph", "Inge"],
                    'phrase': '== Influence == Zhuangzi has influenced '
                                'thinking far beyond East Asia .'
                }
            ]
    }

]

output:
[
    {
        "philosopher" : "philosopher_name",
        "pageid" : id,
        "article" : 
            [
                "frase contenente influence",

                "frase contenente influence" 
            ],

        "rich_article" :
            [
                {
                    'influenced': ['East ', 'Asia '],
                    'influencers': ['Influence ', 'Zhuangzi ',"Plato", "William", "Ralph", "Inge"],
                    'phrase': '== Influence == Zhuangzi has influenced '
                                'thinking far beyond East Asia .'
                }
            ]
        "influenced" :
            [
                "list of influenced after cleaning"
            ],

        "influencers" :
            [
                "list of influencers after cleaning"
            ]
    }

]


                "frase contenente influence" = "Jhon Milton influenced Ghandi and Hannibal Bures' "Power to the Man""
                [
                    list of influencers = ["Jhon", "Milton"] --> "Jhon Milton"
                ],
                [
                    list of influenced = ["Ghandi", "Hannibal", "Bures", "Power", "Man"] --> "Mathma Ghandi", "Hannibal Bures"
                ]
                "him" --> nome del philosopher
"""

phil_name_list = []
for art in jsonlist_no_empty:
    phil_name_list.append(art['philosopher'])


def add_names(philosopher):
    nouns = ["he", "He", "she", "She", "him", "Him", "his", "His", "hers",
             "Hers", "her", "Her", "they", "They", "was", "it", "is"]
    for token_rich in philosopher["rich_article"]:
        for elem in token_rich["influenced"]:
            if elem in nouns:
                token_rich['influenced'].remove(elem)
        for elem in token_rich["influencers"]:
            if elem in token_rich["influencers"]:
                token_rich['influencers'].remove(elem)
    return philosopher



def myfunc(phil):
    mtch_lst = []
    for p in phil_name_list:
        if phil in p:
            mtch_lst.append(p)
    return mtch_lst



def founded_name(name,influencers):
    splittato = name.split() 
    for part_of_name in splittato: #se trovi grazie al nome --> cancella il cognome ancora presente
        try:
            influencers.remove(part_of_name)
        except:
            continue
    return name

def aux_func(lst):
    new_lst = []
    lst = [word for word in lst if word not in stopwords.words('english')]
    for phil in lst:
        if phil in phil_name_list:
            new_lst.append(phil)
            print("cioccato: ",phil)
        else: #nome composto da piu stringhe ?
            possible_names = myfunc(phil) 
            if len(possible_names) == 1: #cioccato
                new_lst.append(founded_name(possible_names[0], lst))
                print("Cioccato subito --> ", possible_names[0])
            else:
                #cerca nomi nella lista per fare nome + cognome
                for name in possible_names:
                    splittato = name.split()
                    try:
                        splittato.remove(phil) #rimuovi da [William, Alston] William
                    except:
                        print("Exclude this phil: ", name)
                    for word in splittato:
                        if word in lst: #if Alston è nella listsa di influencers vuol dire che era William ALston 
                            founded_name(name,lst) 
                            new_lst.append(name)
                            print("aggiunto --> ", name)
    return new_lst      


def reconstruct_phil(output,phil_name_list):
    for match in output:
        match = add_names(match)
        art = match["rich_article"]
        match['influenced'] = [] 
        match['influencers'] = []
        for i in art:
            new_lst_influenced = aux_func(i['influenced'])
            new_lst_influencers = aux_func(i['influencers'])
            new_lst_influenced = [x for x in new_lst_influenced if x != match['philosopher']]
            new_lst_influencers = [x for x in new_lst_influencers if x != match['philosopher']]
            match['influenced'] = match['influenced'] + new_lst_influenced
            match['influencers'] = match['influencers'] + new_lst_influencers
        #aggiungi le tables
        try:
            match['influenced'] = match['influenced'] +  [x for x in match['table']['influenced'] if x in phil_name_list]
        except:
            print("No influenced\n")
        try:
            match['influencers'] = match['influencers'] + [x for x in match['table']['influenced'] if x in phil_name_list]
        except:
            print("No influencers")
        #elimina doppioni
        match['influenced'] = list(set(match['influenced']))
        match['influencers'] = list(set(match['influencers']))
    return output

reconstructed_list = reconstruct_phil(jsonlist_processed,phil_name_list)


#pprint.pprint([x for x in reconstructed_list if x["pageid"] == "21444"])

#pprint.pprint(add_names([x for x in reconstructed_list if x["pageid"] == "21444"][0]))

# ***Pagerank***

In [0]:

def find_indexes_from_names(names, jsonlist):
    indexes = []
    for name in names:
        x = [x["index"] for x in jsonlist if x["philosopher"] == name]
        if len(x) != 0:
            indexes.append(x[0])
    return indexes


edges = []

for i in range(len(jsonlist_processed)):
    jsonlist_processed[i]["index"] = i

for philosopher in jsonlist_processed: 
    influenced_list = find_indexes_from_names(philosopher["influenced"], jsonlist_processed)
    influencers_list = find_indexes_from_names(philosopher["influencers"], jsonlist_processed)
    philosopher["influenced_indexes"] = influenced_list
    philosopher["influencers_indexes"] = influencers_list


edges = []
for philosopher in jsonlist_processed:
    for infed_index in philosopher["influenced_indexes"]:
        edges.append([infed_index, philosopher["index"]])
        
    for infers_index in philosopher["influencers_indexes"]:
        edges.append([philosopher["index"], infers_index])


A = np.array(edges)
weights = [1 for i in range(len(edges))]
G = sparse.csr_matrix((weights, (A[:,0], A[:,1])), shape=(len(jsonlist_processed), len(jsonlist_processed)))

pr=pagerank(G, p=0.85)
result = []
for i in range(len(pr)):
    name = [["Name", x["philosopher"], "Influenced", x["influenced"], "influencers", x["influencers"]] for x in jsonlist_processed if x["index"] == i]
    result.append([pr[i], name[0]])
pprint.pprint(sorted(result, key=lambda x: x[0], reverse = True))


from prettytable import PrettyTable
x = PrettyTable()
x.field_names = ["Philosopher", "Rank", "#Influenced", "#Influencers"]
counter = 1
for i in range(len(pr)):
    x.add_row([jsonlist_processed[i]['philosopher'], pr[i], len(jsonlist_processed[i]['influenced']), len(jsonlist_processed[i]['influencers'])])
x.sortby = "Rank"
x.reversesort = True

print(x)